# Section 0

QPM: Assignment 4

DENG Yunyun <br>
LI Jiaxin <br>
SBAI Ilyas <br>
ZHOU Zhichen <br>

Note: We had to change our database obtained from assignment 3 to get the right format for the stock prices

# Section 0 (4.0)

We retrieve a dataframe ```pricesall``` of the list of the S&P500 adjusted prices that we had saved in an SQL database in the previous assignment.
We then create the dataframe ```prices``` containing only the prices for the relevant 10 stocks.

In [1]:
import pandas as pd
import numpy as np
import sqlite3

# Connecting to the database
SnP500 = sqlite3.connect(
  database="/Users/ilyas/Library/CloudStorage/OneDrive-Personnel/EDHEC/M2/Quantitative Portfolio Management/snp500.sqlite"
)
# Retrieving the price data and setting the date as our index
pricesall = (pd.read_sql_query(
  sql="SELECT * FROM SP500dailyprice",
  con=SnP500,
  parse_dates={"date"})
)
pricesall = pricesall.set_index('date')
pricesall.index = pd.to_datetime(pricesall.index)


In [2]:
# Keeping only the data for the 10 relevant stocks
tic = ['MMM','AOS','ABT','ADM','ADBE','ADP','AES','AFL','A','AKAM']
prices = pricesall[tic]
prices = prices.resample('1M').mean()
prices.head()

,MMM,AOS,ABT,ADM,ADBE,ADP,AES,AFL,A,AKAM
date,,,,,,,,,,
2000-01-31,25.381499,2.318887,8.595265,6.632772,15.531529,24.855753,27.147331,6.522022,41.053272,261.421875
2000-02-29,23.275792,1.952134,8.785981,6.134495,21.961111,21.956365,29.901538,6.222667,52.513095,243.315625
2000-03-31,22.689089,1.809411,8.592946,5.456993,23.301387,23.162192,27.540126,6.302783,74.481565,232.730978
2000-04-30,24.277161,2.076066,10.029198,5.659069,28.146053,24.584584,28.328477,7.361420,56.896264,97.567434
2000-05-31,22.808310,2.356454,10.406937,5.892190,27.954746,25.561754,29.495449,7.621011,47.869098,78.571733


# Section 1

We first define the necessary data: the risk-free rate ```Rf```, the vector of annualised mean returns ```mu```, the variance-covariance matrix of annualised returns ```sigma``` and its inverse ```sigmainv```. <br>
We use the data from the last 60 days.

In [4]:
# Define and compute the necessary data
ones = np.ones((10, 1))
Rf = 0

# Getting the daily returns and computing their mean and covariance matrix (annualised)
ret = pd.DataFrame(prices.pct_change())

mu = (ret.iloc[-60:]*21).mean()
sigma = (ret.iloc[-60:]*21).cov()

# Invert of the covariance matrix
sigmainv = np.linalg.inv(sigma)

We define two functions that we will use later to get the weights of the mean-variance portfolio with a risk-free asset and of the global-minimum-variance portfolio.

In [5]:
def solutionWithRf(mu_targ, Rf, sigma, sigmainv):
    term1Numerator = (mu_targ - Rf)
    #term1Denominator = (mu - Rf*ones).T @ Vinverse @ (mu - Rf*ones)
    term1Denominator = (mu - Rf).T @ sigmainv @ (mu - Rf)
    term1 = term1Numerator/term1Denominator
    #term2 = sigmainv @ (mu - Rf*ones)
    term2 = sigmainv @ (mu - Rf)
    weightsWithRf = term1 * term2
    portfolioVolatilityWithRf = np.sqrt(weightsWithRf.T @ sigma @
    weightsWithRf)
    
    return weightsWithRf

In [6]:
def gmv(sigmainv):
    num = sigmainv @ ones
    denom = ones.T @ sigmainv @ ones
    weights = num/denom
    np.reshape(weights,(10,1))
    return weights

In [9]:
mu_targ = 0.1
meanvar_weights60 = solutionWithRf(mu_targ, Rf, sigma, sigmainv)
gmv_weights60 = gmv(sigmainv)

print('The mean-variance portfolio weights are:',meanvar_weights60)
print('The global-minimum variance portfolio weights are:',gmv_weights60.T )

The mean-variance portfolio weights are: [-2.94503607 -0.41006722  1.3732776   1.87368596  0.3121177   1.61670329
 -0.6811443  -0.88721596  1.96565609 -0.00701816]
The global-minimum variance portfolio weights are: [[ 0.03880817 -0.09094257  0.39835567  0.2915067  -0.05609607  0.16821823
  -0.19011492 -0.01721489  0.29664448  0.16083521]]


# Section 2

We initialise the matrices that will contain the weights for the two strategies at all times.<br>
We loop through the dates, and at each date compute the weights for the two strategies based on the previous 60 days of daily returns.<br>
We arbitrarily chose a mu-targ at 0.1.

In [10]:
weights_meanvar = np.zeros((len(ret)-60,10))
weights_gmv = np.zeros((len(ret)-60,10))
Rf = 0
mu_targ = 0.3

for i in range(len(ret)-60-1):
    periodret = ret.iloc[i:i+60]
    mu = periodret.mean()
    sigma = periodret.cov()
    sigmainv = np.linalg.inv(sigma)
     
    weights_meanvar[i,:] = solutionWithRf(mu_targ, Rf, sigma, sigmainv)
    
    weights_gmv[i:i+1,:] = gmv(sigmainv).T  #i:i+1 not get a shape of (10,1) and not (10,)
    

# Section 3

We compute the out-of-sample returns for both strategy at each date t by multiplying the weights obtained from the stock returns from date t-60 to date t-1 with the stock returns of date t.

In [11]:
# Mean-variance portfolio
ret_meanvar = np.dot(weights_meanvar,(ret.iloc[61:].T))
ret_meanvar = np.sum(ret_meanvar,0)

# Global minimum variance
ret_gmv = np.dot(weights_gmv,(ret.iloc[61:].T))
ret_gmv = np.sum(ret_gmv,0)

# Section 4

We compute both Sharpe returns on annualised data. <br>
The mean-variance portfolio has a higher Sharpe ratio.

In [12]:
Sharpe_meanvar = np.mean(ret_meanvar)*np.sqrt(21)/(np.std(ret_meanvar))
Sharpe_gmv = np.mean(ret_gmv)*np.sqrt(21)/(np.std(ret_gmv))
print('The Sharpe ratio for the mean-variance portfolio is:',Sharpe_meanvar,'\n')
print('The Sharpe ratio for the mean-variance portfolio is:',Sharpe_gmv)

The Sharpe ratio for the mean-variance portfolio is: 1.6275447869560316 

The Sharpe ratio for the mean-variance portfolio is: 1.2969242054114356
